# Project 3


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="moviegenre.png"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Create a solution using with a Machine Learning algorithm - Presentation (5 slides)
- 50% Performance in the Kaggle competition (Normalized acording to class performance in the private leaderboard)


### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [27]:
import os
os.listdir('/content/drive/My Drive/Maestria/Intersemestrales/Modelos avanzados/AdvancedMethodsDataAnalysisClass-master/Exercises') 

['E1 - Examples TSA.md',
 'data_arma.csv',
 'E04 - Panel Data.md',
 '.ipynb_checkpoints',
 'fatalities-from-terrorism (1).csv',
 'size-poverty-gap-world (1).csv',
 'multiTimeline.csv',
 'E02-TSA.ipynb',
 'P1-TSL.md',
 'E03-ARIMA.ipynb',
 'E05-prophet.ipynb',
 'E06 - Decision Trees Overview.md',
 'E10 - microservices.md',
 'E08 - Ensembles Trees Overview.md',
 'E8-RandomForests_Boosting.ipynb',
 'E7-DecisionTrees_Bagging V2.ipynb',
 'P2-UsedVehiclePricePrediction3.ipynb',
 'E8-RandomForests_Boosting_VF.ipynb',
 'model_deployment_proyecto',
 'P2-UsedVehiclePricePrediction.ipynb',
 'P2-UsedVehiclePricePrediction2.ipynb',
 'P2-UsedVehiclePricePredictionCL.ipynb',
 'Tarea NLP catalina lago.docx',
 'E11 - NLP.md',
 'E13-ClassHomeworksAnalysis.ipynb',
 'E12-SentimentPrediction.ipynb',
 'E14-LSTM.ipynb',
 'P3-MovieGenrePrediction.ipynb']

In [28]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [29]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [30]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [31]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


### Create count vectorizer


In [6]:
#vect = CountVectorizer(max_features=1000, stop_words='english')
#X_dtm = vect.fit_transform(dataTraining['plot'])
#X_dtm.shape

In [7]:
#print(vect.get_feature_names()[:50])

In [8]:
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#vect = TfidfVectorizer(stop_words='english')
#X_dtm = vect.fit_transform(dataTraining['plot'])
#features = vect.get_feature_names()
#X_dtm.shape

In [9]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()
import tensorflow_hub as hub


module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)


with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed(dataTraining['plot']))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### Create y

In [32]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [33]:
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [34]:
#X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)
X_train, X_test, y_train_genres, y_test_genres = train_test_split(sentences_embeddings, y_genres, test_size=0.33, random_state=42)

### Train multi-class multi-label model

In [ ]:
print('Cross-validation scores:', scores)
print('Cross-validation accuracy: {:.4f} (+/- {:.4f})'.format(scores.mean(), scores.std() * 2))

In [51]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_depth=None, random_state=42))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                            

In [52]:
y_pred_genres = clf.predict_proba(X_test)

In [53]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8716952349476044

### Predict the testing dataset

In [54]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()
import tensorflow_hub as hub


module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)


with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings_test = session.run(embed(dataTesting['plot']))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [55]:
X_test_dtm = sentences_embeddings_test

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)


In [56]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [57]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,p_Film-Noir,p_History,p_Horror,p_Music,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.055,0.132,0.075,0.038,0.257,0.106,0.008,0.668,0.095,0.156,0.057,0.011,0.087,0.021,0.038,0.200,0.000,0.442,0.044,0.097,0.009,0.218,0.021,0.025
4,0.252,0.067,0.007,0.072,0.385,0.484,0.045,0.526,0.020,0.018,0.053,0.068,0.083,0.018,0.021,0.132,0.006,0.091,0.022,0.008,0.021,0.429,0.097,0.028
5,0.075,0.020,0.001,0.049,0.089,0.747,0.007,0.734,0.005,0.017,0.107,0.033,0.059,0.001,0.003,0.297,0.000,0.094,0.022,0.000,0.001,0.655,0.008,0.007
6,0.239,0.158,0.010,0.056,0.224,0.146,0.026,0.676,0.057,0.045,0.020,0.106,0.139,0.009,0.008,0.146,0.000,0.131,0.118,0.002,0.007,0.373,0.227,0.012
7,0.080,0.046,0.005,0.007,0.135,0.190,0.003,0.331,0.019,0.128,0.022,0.003,0.535,0.003,0.017,0.228,0.000,0.079,0.170,0.000,0.002,0.532,0.002,0.005


In [58]:
res.to_csv('pred_genres_text_RF_1000.csv', index_label='ID')

In [60]:
from google.colab import files
res.to_csv('/content/drive/My Drive/Maestria/Intersemestrales/Modelos avanzados/AdvancedMethodsDataAnalysisClass-master/Exercises/pred_genres_text_RF_1000.csv', index_label='ID')
files.download('pred_genres_text_RF_1000.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>